In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/wing-span-the-bird-species-challenge-depstar-it/submission file.csv
/kaggle/input/wing-span-the-bird-species-challenge-depstar-it/Bird_classififcation_dataset/val/Alpine Swift/aug_624774.jpg
/kaggle/input/wing-span-the-bird-species-challenge-depstar-it/Bird_classififcation_dataset/val/Alpine Swift/Alpine_Swift_0_57_232_556_731_-_f_0561.jpg
/kaggle/input/wing-span-the-bird-species-challenge-depstar-it/Bird_classififcation_dataset/val/Alpine Swift/aug_138409.jpg
/kaggle/input/wing-span-the-bird-species-challenge-depstar-it/Bird_classififcation_dataset/val/Alpine Swift/aug_471977.jpg
/kaggle/input/wing-span-the-bird-species-challenge-depstar-it/Bird_classififcation_dataset/val/Alpine Swift/Alpine_Swift_0_286_95_957_766_-_e_0223.jpg
/kaggle/input/wing-span-the-bird-species-challenge-depstar-it/Bird_classififcation_dataset/val/Alpine Swift/Alpine_Swift_0_96_58_211_174_-_g_0302.jpg
/kaggle/input/wing-span-the-bird-species-challenge-depstar-it/Bird_classififcation_dataset/val/Al

In [2]:
import os
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from PIL import Image
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset

In [3]:
base_dir = '/kaggle/input/wing-span-the-bird-species-challenge-depstar-it/Bird_classififcation_dataset'  
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

In [4]:
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [5]:
val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [13]:
train_data = datasets.ImageFolder(train_dir, transform=train_transform)
val_data = datasets.ImageFolder(val_dir, transform=val_test_transform)

train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
val_loader = DataLoader(val_data, batch_size=128, shuffle=False)

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet18(weights=None)
model.fc = nn.Linear(model.fc.in_features, len(train_data.classes))
model = model.to(device)

In [15]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [16]:
epochs = 10
for epoch in range(epochs):
    model.train()
    total_loss = 0
    print(f"\n Epoch {epoch+1}/{epochs}")
    for batch_idx, (imgs, labels) in enumerate(train_loader):
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        print(f"  🔸 Batch {batch_idx+1}/{len(train_loader)} - Loss: {loss.item():.4f}")
    
    print(f" Epoch {epoch+1} completed - Total Training Loss: {total_loss:.4f}")


 Epoch 1/10
  🔸 Batch 1/101 - Loss: 1.7761
  🔸 Batch 2/101 - Loss: 1.4666
  🔸 Batch 3/101 - Loss: 1.4399
  🔸 Batch 4/101 - Loss: 1.3978
  🔸 Batch 5/101 - Loss: 1.2335
  🔸 Batch 6/101 - Loss: 1.2188
  🔸 Batch 7/101 - Loss: 1.0339
  🔸 Batch 8/101 - Loss: 1.0770
  🔸 Batch 9/101 - Loss: 0.9595
  🔸 Batch 10/101 - Loss: 1.0308
  🔸 Batch 11/101 - Loss: 1.0407
  🔸 Batch 12/101 - Loss: 1.0664
  🔸 Batch 13/101 - Loss: 1.0018
  🔸 Batch 14/101 - Loss: 0.9799
  🔸 Batch 15/101 - Loss: 0.9635
  🔸 Batch 16/101 - Loss: 0.9230
  🔸 Batch 17/101 - Loss: 0.8877
  🔸 Batch 18/101 - Loss: 0.9180
  🔸 Batch 19/101 - Loss: 0.8407
  🔸 Batch 20/101 - Loss: 0.9337
  🔸 Batch 21/101 - Loss: 0.8440
  🔸 Batch 22/101 - Loss: 0.9084
  🔸 Batch 23/101 - Loss: 0.8511
  🔸 Batch 24/101 - Loss: 0.8254
  🔸 Batch 25/101 - Loss: 0.9443
  🔸 Batch 26/101 - Loss: 0.8883
  🔸 Batch 27/101 - Loss: 0.7397
  🔸 Batch 28/101 - Loss: 0.7638
  🔸 Batch 29/101 - Loss: 0.7732
  🔸 Batch 30/101 - Loss: 0.8595
  🔸 Batch 31/101 - Loss: 0.9895
  🔸 

In [17]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for imgs, labels in val_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
print(f"Validation Accuracy: {100 * correct / total:.2f}%")

Validation Accuracy: 93.03%


In [18]:
class TestDataset(Dataset):
    def __init__(self, path, transform=None):
        self.files = sorted(os.listdir(path))
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        img_name = self.files[idx]
        img_path = os.path.join(self.path, img_name)
        img = Image.open(img_path).convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img_name, img

In [19]:
test_data = TestDataset(test_dir, transform=val_test_transform)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

In [31]:
import torch.nn.functional as F

model.eval()
results = []

with torch.no_grad():
    for batch in test_loader:
        names, imgs = batch
        imgs = imgs.to(device)

        outputs = model(imgs)
        probs = F.softmax(outputs, dim=1)
        _, preds = torch.max(probs, 1)

        for name, pred in zip(names, preds):
            results.append((name, train_data.classes[pred.item()]))

# ✅ Update column names: 'image_name' and 'class_name'
submission = pd.DataFrame(results, columns=["image_name", "class_name"])
submission.to_csv("/kaggle/working/submission.csv", index=False)
print("Submission file saved to /kaggle/working/submission.csv")

Submission file saved to /kaggle/working/submission.csv
